# <center> webscraping

### première partie du projet bloc 1

- Webscraping du site [www.allocine.fr](https://www.allocine.fr/films/)

![filtres](images/filtresSMALL.png)

## Sources :
[beautiful-soup-4](https://beautiful-soup-4.readthedocs.io/en/latest/)<br>
<br>
<br>
<br>


In [ ]:
%reset

In [1]:
import os
import re
#import httpx
import requests
import pandas as pd

from bs4 import BeautifulSoup
from IPython.display import display
from tqdm import tqdm

%config IPCompleter.greedy = True

url_site = 'https://www.allocine.fr/films/'

### On scrape tous les genres de film

In [2]:
# Scrap all categories
r = requests.get(url_site, auth=('user', 'pass'))
if r.status_code != 200:
    print("url_site error")
    
soup = BeautifulSoup(r.content, 'html.parser')
print(type(soup))

categories = []
eltCategories = soup.find('div', class_='filter-entity-section')
for elt in eltCategories.find_all('li'):
    categories.append(elt.a.text)

df_categories = pd.Series(categories)

dict_n_cat = {k:v for k, v in enumerate(categories)}
dict_cat_n = {v:k for v, k in dict_n_cat.items()}

<class 'bs4.BeautifulSoup'>


### On scrapte les films par période
[1980 - 1989] puis [1990 - 1999] ...

In [16]:
# Scrap url of years we want to scrap the movies
# Not Working
# I cannot get the url by scraping
decades_to_scrap = ['1980 - 1989']
eltYears = eltCategories.find_next_sibling()
lstUrl = []
eltCur = eltYears.find('li')

while eltCur:
    text = eltCur.span.text
    # print(eltCur)
    # print(eltCur.span.get('href'))
    if text in decades_to_scrap:
        print("**", text)
    else:
        print(eltCur.span.text)
    eltCur = eltCur.find_next_sibling()

2030 - 2039
2020 - 2029
2010 - 2019
2000 - 2009
1990 - 1999
** 1980 - 1989
1970 - 1979
1960 - 1969
1950 - 1959
1940 - 1949
1930 - 1939
1920 - 1929
1910 - 1919
1900 - 1909
1890 - 1899


### Ou bien nous pouvons entrer les url manuellement

In [17]:
url_decades = url_site + 'decennie-1980/'
url_year = url_site + 'decennie-1980/annee-1980/'

def getNumberOfPages(elt):
    nb = 0
    while elt:
        if elt.text.isdigit():
            nb = elt.text
        elt = elt.find_next_sibling()
    return int(nb)

r = requests.get(url_year, auth=('user', 'pass'))
if r.status_code != 200:
    print("url_site error")

# We get the number of pages for this year
soup = BeautifulSoup(r.content, 'html.parser')
pagination = soup.find('div', class_='pagination-item-holder')
nb_pages = int(pagination.find_all('span')[-1].text)
assert nb_pages == getNumberOfPages(pagination.find('span'))
print(nb_pages)

47


### On scrape chaque page de films pour une année donnée

In [34]:
def scrapMoviePage(soupMovies):
    #print(soupMovies)
    elt = soupMovies.find('li', class_='mdl')
    elt = elt.find("h2")
#    print(elt)
    url_movie = elt.find('a').get('href')
#    print(url_movie)
    print(elt.text)
    
n_test = 3
for i in range(1, n_test):
    url_year_page = url_year + '?page=' + str(i)
    #print(url_year_page)
    r = requests.get(url_year_page, auth=('user', 'pass'))
    if r.status_code != 200:
        print("url_site error")        
    soupMovies = BeautifulSoup(r.content, 'html.parser')
    scrapMoviePage(soupMovies)


Shining


La Chasse - Cruising



### On scrape les pays

In [ ]:
eltCountries = eltYears.find_next_sibling()
print(eltCountries)